In [47]:
import numpy as np
import pandas as pd
from IPython.core.debugger import set_trace
import pickle

In [53]:
items = 'pine oak rose daisy robin canary sunfish salmon'.split() # just copied from the table
nitem=len(items)


In [54]:
# goal: 1) dictionary between index and label 2) triplets
keys = ['item','relation','outp']

TripletD = {}
for k in keys:
    TripletD[k]=[]

# start reading from table: first part
nitm_tb1 = 5
f = open("rawdata_pdf_0-5.txt")
content = f.readline().split()

relas = []
relaind = -1
relarow =[] # record each row's corresponding relations for reading the 2nd part of table

outps =[]
outind = -1
itemind = -1

for c in content:
    if '...' in c: #relation name
        relas.append(c.strip('...'))
        relaind += 1
        continue
    if '1' in c or '0' in c: # yes or no about each triplet. 
        if len(c)>1: # read one-by-one
            for itemind in range(len(c)):
                if c[itemind]=='1':
                    write_triplet(itemind,relaind,outind)
            continue
        else:
            itemind = (itemind+1)%nitm_tb1
            if c=='1': # not recording 0s. do you want negative training set as well??
                write_triplet(itemind,relaind,outind)            
            continue
    else: # outp. marks a new row
        outps.append(c)
        outind+=1
        relarow.append(relaind) # what's the current relation in this row
        continue
            
        

# the 2nd part of table
nitm_tb2 = 3
f = open("rawdata_pdf_5-7.txt")
content = f.readline().split()

k=-1
for c in content:
    k+=1
    if c=='1':
        itemind=k%nitm_tb2+nitm_tb1
        outind = int(k/nitm_tb2)
        relaind = relarow[outind]
        write_triplet(itemind,relaind,outind)    

        
IndexDict = {} 
IndexDict['item'] = pd.Series(data=items)
IndexDict['relation'] = pd.Series(data=relas)
IndexDict['outp']=pd.Series(data=outps)


In [55]:
# convert into 1-hot representation
Triplet1hots={}
for key in TripletD.keys():
    Triplet1hots[key] = ind2onehot(TripletD[key],len(IndexDict[key]))

In [58]:
pickle.dump([Triplet1hots,TripletD,IndexDict],open('tableB1','wb'))

In [40]:
def ind2onehot(inds,maxn):
    onehot = np.zeros((len(inds),maxn))
    for i in range(len(inds)):
        onehot[i][inds[i]]=1
    return onehot
def onehot2ind(onehots):
    inds = np.zeros(len(onehots),dtype=int)
    for k in range(len(onehots)):
        oh = onehots[k]
        inds[k] = np.argwhere(oh)
    return inds

In [15]:
def write_triplet(item,rela,outp): # not transformed into 1-hot.
    TripletD['item'].append(item)
    TripletD['relation'].append(rela)
    TripletD['outp'].append(outp)